In [1]:
# with open("input.txt", "rt") as f:
#     map_ = []
#     for row in f.read().strip().split("\n"):
#         map_.append("." + row + ".")
#     map_.insert(0, "." * len(map_[0]))
#     map_.append("." * len(map_[0]))

In [2]:
from collections import namedtuple

In [3]:
with open("input.txt", "rt") as f:
    sketch = f.read().strip().split("\n")
sketch = [f".{row}." for row in sketch]

n_rows = len(sketch)
n_cols = len(sketch[0])

padding_row = "." * n_cols
sketch.insert(0, padding_row)
sketch.append(padding_row)

# Point def and lookup helpers

Point = namedtuple("Point", ["x", "y"])


def top(p: Point) -> Point:
    return Point(p.x, p.y - 1)


def left(p: Point) -> Point:
    return Point(p.x - 1, p.y)


def right(p: Point) -> Point:
    return Point(p.x + 1, p.y)


def bottom(p: Point) -> Point:
    return Point(p.x, p.y + 1)


def lookup_sketch(p: Point) -> str:
    return sketch[p.y][p.x]


# Find starting point

start: Point = None
for y, row in enumerate(sketch):
    if "S" in row:
        x = row.index("S")
        start = Point(x, y)

# Replace it in the sketch with appropriate pipe element (required for part 2)

print(f"{start = }")
print(f"{lookup_sketch(start) = }")

t = lookup_sketch(top(start))
l = lookup_sketch(left(start))
r = lookup_sketch(right(start))
b = lookup_sketch(bottom(start))

print(f"{t = }")
print(f"{l = }")
print(f"{r = }")
print(f"{b = }")

replacements = set("7FJL-|")

if t in "7|F":
    replacements.discard("F")
    replacements.discard("-")
    replacements.discard("7")
if l in "L-F":
    replacements.discard("F")
    replacements.discard("|")
    replacements.discard("L")
if r in "J-7":
    replacements.discard("7")
    replacements.discard("|")
    replacements.discard("J")
if b in "J|L":
    replacements.discard("J")
    replacements.discard("-")
    replacements.discard("L")

assert len(replacements) == 1

sketch[start.y] = sketch[start.y].replace("S", replacements.pop())
print(f"{lookup_sketch(start) = }")

start = Point(x=18, y=36)
lookup_sketch(start) = 'S'
t = '7'
l = '.'
r = '|'
b = 'J'
lookup_sketch(start) = '|'


# Part 1

In [4]:
t = lookup_sketch(top(start))
l = lookup_sketch(left(start))
r = lookup_sketch(right(start))
b = lookup_sketch(bottom(start))

# Find a point we can go to from start

if t in "7|F":
    current_point = top(start)
    direction = "t"
elif l in "L-F":
    current_point = left(start)
    direction = "l"
elif r in "J-7":
    current_point = right(start)
    direction = "r"
elif b in "J|L":
    current_point = bottom(start)
    direction = "b"
else:
    raise Exception()

# Look for next points until loop is closed

loop = [start]
while current_point != start:
    loop.append(current_point)

    # Go next

    char = lookup_sketch(current_point)

    if direction == "t":
        if char == "7":
            current_point, direction = left(current_point), "l"
        elif char == "|":
            current_point, direction = top(current_point), "t"
        elif char == "F":
            current_point, direction = right(current_point), "r"
        else:
            raise Exception()
    
    elif direction == "l":
        if char == "L":
            current_point, direction  = top(current_point), "t"
        elif char == "-":
            current_point, direction  = left(current_point), "l"
        elif char == "F":
            current_point, direction  = bottom(current_point), "b"
        else:
            raise Exception()
    
    elif direction == "r":
        if char == "J":
            current_point, direction  = top(current_point), "t"
        elif char == "-":
            current_point, direction  = right(current_point), "r"
        elif char == "7":
            current_point, direction  = bottom(current_point), "b"
        else:
            raise Exception()
    
    elif direction == "b":
        if char == "J":
            current_point, direction  = left(current_point), "l"
        elif char == "|":
            current_point, direction  = bottom(current_point), "b"
        elif char == "L":
            current_point, direction  = right(current_point), "r"

    else:
        raise Exception()

len(loop) // 2

6897

# Part 2

In [5]:
{Point(0,0)}

{Point(x=0, y=0)}

In [6]:
from itertools import pairwise

In [7]:
neighbors_f = (
    ("t", top),
    ("l", left),
    ("r", right),
    ("b", bottom),
)

visited = [[False] * n_cols for _ in range(n_rows)]
outside = {Point(0, 0)}  # (0, 0) is part of padding - definitely outside

to_check = [Point(0, 0)]
while to_check:
    point = to_check.pop()

    for direction, f in neighbors_f:
        neighbor = f(point)

        if (
            neighbor.x < 0
            or neighbor.y < 0
            or neighbor.x >= n_cols
            or neighbor.y >= n_rows
            or visited[neighbor.y][neighbor.x]
        ):
            continue

        visited[neighbor.y][neighbor.x] = True

        # If a point is a neigbor of a point outside the loop
        # and it's not a part of the loop, then it's also outside
        if neighbor not in loop:
            outside.add(neighbor)
            to_check.append(neighbor)
        # If we hit a point that is part of the loop, we will
        # go along the loop counterclockwise and detect any
        # points outside the loop that are directly next to it
        else:
            if neighbor == loop[-1]:
                sliced_loop = loop
            else:
                idx = loop.index(neighbor)
                sliced_loop = loop[idx:] + loop[:idx] + [neighbor]

            # Make sure loop is sorted clockwise
            next_point = sliced_loop[0]
            prev_point = sliced_loop[-2]
            if not (
                # fmt: off
                   direction == "t" and (next_point == right(neighbor)  or prev_point == left(neighbor))
                or direction == "l" and (next_point == top(neighbor)    or prev_point == bottom(neighbor))
                or direction == "r" and (next_point == bottom(neighbor) or prev_point == top(neighbor))
                or direction == "b" and (next_point == left(neighbor)   or prev_point == right(neighbor))
                # fmt: on
            ):
                sliced_loop = list(reversed(sliced_loop))
            
            # Traverse the loop
            for prev_point, loop_point in pairwise(sliced_loop):
                visited[loop_point.y][loop_point.x] = True

                # Figure out the direction
                if loop_point.y < prev_point.y:
                    loop_direction = "t"
                elif loop_point.y > prev_point.y:
                    loop_direction = "b"
                elif loop_point.x < prev_point.x:
                    loop_direction = "l"
                elif loop_point.x > prev_point.x:
                    loop_direction = "r"
                else:
                    raise Exception()

                # Find points to add

                char = lookup_sketch(loop_point)
                near_loop = []

                if loop_direction == "t":
                    if char == "7":
                        near_loop.append(top(loop_point))
                        near_loop.append(right(loop_point))
                    elif char == "|":
                        near_loop.append(right(loop_point))
                    elif char == "F":
                        pass
                    else:
                        raise Exception()
                
                elif loop_direction == "l":
                    if char == "L":
                        pass
                    elif char == "-":
                        near_loop.append(top(loop_point))
                    elif char == "F":
                        near_loop.append(top(loop_point))
                        near_loop.append(left(loop_point))
                    else:
                        raise Exception()
                
                elif loop_direction == "r":
                    if char == "J":
                        near_loop.append(right(loop_point))
                        near_loop.append(bottom(loop_point))
                    elif char == "-":
                        near_loop.append(bottom(loop_point))
                    elif char == "7":
                        pass
                    else:
                        raise Exception()

                elif loop_direction == "b":
                    if char == "J":
                        pass
                    elif char == "|":
                        near_loop.append(left(loop_point))
                    elif char == "L":
                        near_loop.append(left(loop_point))
                        near_loop.append(bottom(loop_point))
                    else:
                        raise Exception()
                
                else:
                    raise Exception()
                
                for near_loop_point in near_loop:
                    if near_loop_point not in loop:
                        outside.add(near_loop_point)
                        to_check.append(near_loop_point)

n_rows * n_cols - (len(outside) +len(loop))

367

## Solution with scikit-image

`> python -m pip install -U scikit-image`


In [8]:
import numpy as np
from skimage.draw import polygon

In [9]:
vertices = np.array(loop)


img = np.zeros((n_cols, n_rows), 'uint8')
rr, cc = polygon(vertices[:,1], vertices[:,0], img.shape)
img[rr,cc] = 1

int(img.sum() - len(loop))

367